# Training on an Advanced Standard CNN Architecture
* https://keras.io/applications/
* The 9 Deep Learning Papers You Need To Know About: https://adeshpande3.github.io/adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html

[Neural Network Architectures](https://medium.com/towards-data-science/neural-network-architectures-156e5bad51ba)
![Performance of CNN Architectures](https://cdn-images-1.medium.com/max/1600/1*kBpEOy4fzLiFxRLjpxAX6A.png)

top-1 rating on ImageNet: https://stats.stackexchange.com/questions/156471/imagenet-what-is-top-1-and-top-5-error-rate

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import matplotlib.pylab as plt
import numpy as np

In [6]:
from distutils.version import StrictVersion

In [7]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.18.1


In [8]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.2.1


In [9]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

Using TensorFlow backend.


2.0.8


In [10]:
import pandas as pd
print(pd.__version__)

assert StrictVersion(pd.__version__) >= StrictVersion('0.20.0')

0.20.1


## Preparation

In [52]:
# for VGG, ResNet, and MobileNet
INPUT_SHAPE = (224, 224)

# for InceptionV3, InceptionResNetV2, Xception
# INPUT_SHAPE = (299, 299)

In [53]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, INPUT_SHAPE) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [54]:
# Load datasets.
ROOT_PATH = "./"
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
original_images, original_labels = load_data(original_dir, type=".ppm")

In [55]:
X, y = original_images, original_labels

### Uncomment next three cells if you want to train on augmented image set
#### Otherwise Overfitting can not be avoided because image set is simply too small

In [56]:
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/augmented-signs.zip
# from zipfile import ZipFile
# zip = ZipFile('augmented-signs.zip')
# zip.extractall('.')

In [57]:
data_dir = os.path.join(ROOT_PATH, "augmented-signs")
augmented_images, augmented_labels = load_data(data_dir, type=".png")

In [58]:
# merge both data sets

all_images = np.vstack((X, augmented_images))
all_labels = np.vstack((y, augmented_labels))

# shuffle
# https://stackoverflow.com/a/4602224

p = numpy.random.permutation(len(all_labels))
shuffled_images = all_images[p]
shuffled_labels = all_labels[p]
X, y = shuffled_images, shuffled_labels

### Split test and train data 80% to 20%

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape

((3335, 224, 224, 3), (3335, 6))

# Training Xception
* Slighly optimized version of Inception: https://keras.io/applications/#xception
* Inception V3 no longer using non-sequential tower architecture, rahter short cuts: https://keras.io/applications/#inceptionv3
* Uses Batch Normalization:
  * https://keras.io/layers/normalization/#batchnormalization
  * http://cs231n.github.io/neural-networks-2/#batchnorm
  * Batch Normalization still exist even in prediction model
  * normalizes activations for each batch around 0 and standard deviation close to 1
  * replaces Dropout except for final fc layers
    * as a next step might make sense to alter classifier to again have Dropout for training
* All that makes it ideal for our use case

![Inception V3](https://djcordhose.github.io/ai/img/architectures/inception_v3_architecture.png)

In [22]:
from keras.applications.xception import Xception

model = Xception(classes=6, weights=None)

In [23]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [24]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# !rm -rf ./tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=./tf_log
# open http://localhost:6006

## This is a truly complex model
### Batch size needs to be small overthise model does not fit in memory
### Will take long to train, even on GPU
### on augmented dataset 4 minutes on K80 per Epoch: 400 Minutes for 100 Epochs = 6-7 hours

In [31]:
# Depends on harware GPU architecture, model is really complex, batch needs to be small (this works well on K80)
BATCH_SIZE = 25

In [50]:
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
%time model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[tb_callback, early_stopping_callback], batch_size=BATCH_SIZE)

Train on 2668 samples, validate on 667 samples
Epoch 1/50
2668/2668 [==============================] - 228s - loss: 1.5403 - acc: 0.3827 - val_loss: 1.7899 - val_acc: 0.2864
Epoch 2/50
2668/2668 [==============================] - 227s - loss: 0.7724 - acc: 0.7043 - val_loss: 1.8531 - val_acc: 0.1634
Epoch 3/50
2668/2668 [==============================] - 227s - loss: 0.3242 - acc: 0.8849 - val_loss: 2.0106 - val_acc: 0.1634
Epoch 4/50
2668/2668 [==============================] - 227s - loss: 0.1614 - acc: 0.9442 - val_loss: 1.0445 - val_acc: 0.6732
Epoch 5/50
2668/2668 [==============================] - 227s - loss: 0.1077 - acc: 0.9636 - val_loss: 1.8615 - val_acc: 0.6672
Epoch 6/50
2668/2668 [==============================] - 228s - loss: 0.0941 - acc: 0.9696 - val_loss: 0.5575 - val_acc: 0.8471
Epoch 7/50
2668/2668 [==============================] - 228s - loss: 0.0389 - acc: 0.9903 - val_loss: 0.2147 - val_acc: 0.9370
Epoch 8/50
2668/2668 [==============================] - 227s - l

## Each Epoch takes very long
## Extremely impressing how fast it converges: Almost 100% for validation starting from epoch 25

## TODO: Metrics for Augmented Data

### Accuracy
![Accuracy Xception](https://djcordhose.github.io/ai/img/tensorboard/cnn-acc-xception.png)
### Validation Accuracy
![Validation Accuracy Xception](https://djcordhose.github.io/ai/img/tensorboard/cnn-val-acc-xception.png)

In [34]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

3335/3335 [==============================] - 81s    


(0.0052181122955644744, 0.99850074882092688)

In [35]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

834/834 [==============================] - 20s    


(0.01086848787854011, 0.99640287576819497)

In [36]:
original_loss, original_accuracy = model.evaluate(original_images, original_labels, batch_size=BATCH_SIZE)
original_loss, original_accuracy

379/379 [==============================] - 9s     


(0.00010616126536950952, 1.0)

In [37]:
model.save('xception-augmented.hdf5')

## Alternative: ResNet
* basic ideas
  * depth does matter
  * 8x deeper than VGG
  * possible by using shortcuts and skipping final fc layer
* https://keras.io/applications/#resnet50
* https://medium.com/towards-data-science/neural-network-architectures-156e5bad51ba

http://arxiv.org/abs/1512.03385
![Deep Learning](https://raw.githubusercontent.com/DJCordhose/ai/master/docs/img/residual.png)

In [39]:
from keras.applications.resnet50 import ResNet50

model = ResNet50(classes=6, weights=None)

In [40]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_2[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [41]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [42]:
# !rm -rf ./tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=./tf_log
# open http://localhost:6006

In [63]:
# Depends on harware GPU architecture, model is really complex, batch needs to be small (this works well on K80)
BATCH_SIZE = 50

In [ ]:
# https://github.com/fchollet/keras/issues/6014
# batch normalization seems to mess with accuracy when test data set is small, accuracy here is different from below
%time model.fit(X_train, y_train, epochs=50, validation_split=0.2, batch_size=BATCH_SIZE, callbacks=[tb_callback, early_stopping_callback])

Train on 2668 samples, validate on 667 samples
Epoch 1/50
2668/2668 [==============================] - 100s - loss: 2.2870 - acc: 0.2804 - val_loss: 13.1700 - val_acc: 0.1829
Epoch 2/50
2668/2668 [==============================] - 98s - loss: 2.0983 - acc: 0.2995 - val_loss: 11.2897 - val_acc: 0.1784
Epoch 3/50
2668/2668 [==============================] - 98s - loss: 1.9620 - acc: 0.2969 - val_loss: 10.2811 - val_acc: 0.1709
Epoch 4/50
2668/2668 [==============================] - 98s - loss: 1.7103 - acc: 0.3283 - val_loss: 3.5921 - val_acc: 0.1574
Epoch 5/50
2668/2668 [==============================] - 98s - loss: 1.7245 - acc: 0.3415 - val_loss: 3.3466 - val_acc: 0.1889
Epoch 6/50
2668/2668 [==============================] - 98s - loss: 1.5632 - acc: 0.3632 - val_loss: 2.2134 - val_acc: 0.2039
Epoch 7/50
2668/2668 [==============================] - 98s - loss: 1.4740 - acc: 0.4363 - val_loss: 1.8380 - val_acc: 0.2489
Epoch 8/50
2668/2668 [==============================] - 98s - loss:

## X for validation

## TODO: Metrics for Augmented Data

### Accuracy
![Accuracy ResNet](https://djcordhose.github.io/ai/img/tensorboard/cnn-acc-resnet.png)
### Validation Accuracy
![Validation Accuracy ResNet](https://djcordhose.github.io/ai/img/tensorboard/cnn-val-acc-resnet.png)

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

In [ ]:
original_loss, original_accuracy = model.evaluate(original_images, original_labels, batch_size=BATCH_SIZE)
original_loss, original_accuracy